In [ ]:
#!pip install pyyaml==5.1
import torch
import torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version

In [ ]:
print(torch.cuda.get_device_properties(0).total_memory/1e9)
print(torch.cuda.memory_reserved(0)/1e9)
print(torch.cuda.memory_allocated(0)/1e9)

In [ ]:
#%matplotlib notebook
import glob
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from IPython.display import Image
import time

from shapely.geometry import Polygon

# importing required libraries
import pandas as pd
import matplotlib.patches as patches
import matplotlib.pyplot as plt
import pandas as pd
from ..utils.get_iou import get_max_iou, get_iou, get_overlap
import pickle
import random
import json
import os
from collections import Counter
import cv2
import numpy as np

import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
# import some common detectron2 utilities
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2 import model_zoo
from detectron2.structures import BoxMode
from detectron2.utils.visualizer import ColorMode
from detectron2.utils.visualizer import Visualizer
from detectron2.data import DatasetCatalog
from detectron2.structures import BoxMode
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator
from detectron2.engine import DefaultTrainer

from os import path

In [ ]:
from ..utils.metric_tablebank_my_implementation import metric_table_bank_union
from ..utils.get_models_and_set import get_train_data_word, get_train_data_latex, get_train_data_publaynet, get_train_data_word_pln, get_train_data_latex_pln, \
                                    get_train_data_pln_and_empty, get_train_data_and_word_pln_empty, get_train_data_and_latex_pln_empty, get_train_data_word_and_latex_pln_empty, \
                                    get_test_data_word, get_valid_data, get_train_data_latex, get_train_data_word, get_test_data_latex, get_test_data_publaynet, get_train_data_pln_latex_word, \
                                    get_test_data_both

# Get data

In [ ]:
path_data = "/data/rali5/Tmp/yockelle/TableBank/TableBank/Detection/"

In [ ]:
train_set_name = "my_dataset_train"
valid_set_name = "my_dataset_valid"
test_set_name = "my_dataset_test"

# Train set
DatasetCatalog.register(train_set_name, get_train_data_latex)
MetadataCatalog.get(train_set_name).set(thing_classes=["table"])
text_metadata_train = MetadataCatalog.get(train_set_name)

# Validation set
DatasetCatalog.register(valid_set_name, get_valid_data)
MetadataCatalog.get(valid_set_name).set(thing_classes=["table"])
text_metadata_valid = MetadataCatalog.get(valid_set_name)

# Test set
DatasetCatalog.register(test_set_name, get_test_data_both)
MetadataCatalog.get(test_set_name).set(thing_classes=["table"])
text_metadata_test = MetadataCatalog.get(test_set_name)

In [ ]:
class CocoTrainer(DefaultTrainer):

  @classmethod
  def build_evaluator(cls, cfg, dataset_name, output_folder=None):

    if output_folder is None:
        os.makedirs("coco_eval", exist_ok=True)
        output_folder = "coco_eval"

    return COCOEvaluator(dataset_name, cfg, False, output_folder)

In [ ]:
cfg = get_cfg()
cfg.merge_from_file("/data/rali5/Tmp/yockelle/TableBank/TableBank/output/latex/X152/config_latex_X152.yaml")
cfg.DATASETS.TRAIN = (train_set_name,)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 4
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "/data/rali5/Tmp/yockelle/TableBank/TableBank/output/latex/X152/model_final_latex_X152.pth")
#cfg.SOLVER.IMS_PER_BATCH = 4
#cfg.SOLVER.BASE_LR = 0.0125  # pick a good LR
#cfg.SOLVER.BASE_LR = 0.0125  # pick a good LR
#cfg.SOLVER.MAX_ITER = 3000    # 300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
#cfg.SOLVER.STEPS = []        # do not decay learning rate
#cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # faster, and good enough for this toy dataset (default: 512)
#cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 256
#cfg.MODEL.PANOPTIC_FPN.COMBINE.INSTANCES_CONFIDENCE_THRESH = 0.5
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
# NOTE: this config means the number of classes, but a few popular unofficial tutorials incorrect uses num_classes+1 here.
cfg.TEST.EVAL_PERIOD = 1000
CUDA_LAUNCH_BLOCKING = 1

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
#trainer = DefaultTrainer(cfg) 
trainer = CocoTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

In [ ]:
"""
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.3   # set the testing threshold for this model
cfg.DATASETS.TEST = (test_set_name, )
predictor = DefaultPredictor(cfg)
"""

cfg = get_cfg()
cfg.merge_from_file("/data/rali5/Tmp/yockelle/TableBank/TableBank/output/latex/X152/config_latex_X152.yaml")
cfg.DATASETS.TRAIN = (train_set_name,)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 4
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "/data/rali5/Tmp/yockelle/TableBank/TableBank/output/latex/X152/model_final_latex_X152.pth")
cfg.SOLVER.IMS_PER_BATCH = 4
#cfg.SOLVER.BASE_LR = 0.0125  # pick a good LR
#cfg.SOLVER.MAX_ITER = 7000    # 300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
#cfg.SOLVER.STEPS = []        # do not decay learning rate
#cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 256
cfg.MODEL.PANOPTIC_FPN.COMBINE.INSTANCES_CONFIDENCE_THRESH = 0.5
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
# NOTE: this config means the number of classes, but a few popular unofficial tutorials incorrect uses num_classes+1 here.
cfg.TEST.EVAL_PERIOD = 1000
CUDA_LAUNCH_BLOCKING = 1
predictor = DefaultPredictor(cfg)

In [ ]:
test_data = get_test_data_word()
threshold = 0.975

precision_tab = []
recall_tab = []

sum_numerator = 0
sum_numerator_old = 0
sum_denominator_precision = 0
sum_denominator_recall = 0

test_count = 0

for i in range(len(test_data)):
  if i%1000 == 0:
    print(i)
  image_dict = test_data[i]
  image_name = "/data/rali5/Tmp/yockelle/TableBank/" + image_dict["file_name"]
  im = cv2.imread(image_name)
  outputs = predictor(im)

  predictions_detectron = outputs["instances"].pred_boxes.tensor.cpu().numpy()
  scores_detectron = outputs["instances"].scores.cpu().numpy()
  predictions_with_threshold = []
  for j, p in enumerate(predictions_detectron):
    if scores_detectron[j]>threshold:
      predictions_with_threshold.append(p)
  predictions_with_threshold = np.array(predictions_with_threshold)

  # Get the ground truth
  bbox_ground_truth = []
  truth = image_dict["annotations"]
  for t in truth:
      x_min = t["bbox"][0]
      y_min = t["bbox"][1]
      w = t["bbox"][2]
      h = t["bbox"][3]
      x_max = x_min + w
      y_max = y_min + h
      bbox_ground_truth.append([x_min, y_min, x_max, y_max])

  result = metric_table_bank_union(bbox_ground_truth, predictions_with_threshold)

  if result != None:
    test_count += 1
    numerator, denominator_precision, denominator_recall, old_way_area_union = result

    sum_numerator += numerator
    sum_numerator_old += old_way_area_union
    sum_denominator_precision += denominator_precision
    sum_denominator_recall += denominator_recall

precision = sum_numerator/sum_denominator_precision
recall = sum_numerator/sum_denominator_recall
f1 = (2 * precision * recall) / (precision + recall)

print("Precision ", precision)
print("Recall ", recall)
print("F1 ", f1)

In [ ]:
test_latex_predictions = {}
test_latex_truth = {}

test_data = get_test_data_both() + get_test_data_publaynet()

count_pred = 0
count_truth = 0

all_image_preds = []
all_annotations_preds = []

all_image_truth = []
all_annotations_truth = []

sum_numerator = 0
sum_numerator_old = 0
sum_denominator_precision = 0
sum_denominator_recall = 0

for index, image_dict in enumerate(test_data):
  one_image_pred = {}
  one_image_truth = {}
  
  if index%500 == 0:
    print(index)

  try: 
    image_name = "/data/rali5/Tmp/yockelle/TableBank/" + image_dict["file_name"]  
    im = cv2.imread(image_name).shape
  except:
    image_name = image_dict["file_name"]
    im = cv2.imread(image_name).shape
  #image_name = "/data/rali5/Tmp/yockelle/PubLayNet/" + image_dict["file_name"]
  #image_name = image_dict["file_name"]
  only_name = image_dict["file_name"].split("/")[-1]

  truths_boxes = []
  preds_boxes = []

  problem = False

  # ADD TRUTH
  for annot in image_dict["annotations"]:
    #if scores_detectron[j]>threshold:
    p = annot["bbox"]
    x1 = p[0]
    y1 = p[1]
    w = p[2]
    h = p[3]
    x2 = p[0] + w
    y2 = p[1] + h
    if y1 == y2 or y2<y1:
      #print("here")
      #print(count_truth)
      problem = True
      break

    if not(problem):
      
      one_annotations = {}
      one_annotations["id"] = count_truth
      one_annotations["segmentation"] = []
      one_annotations["area"] = float(w * h)
      one_annotations["iscrowd"] = 0
      one_annotations["ignore"] = 0
      one_annotations["image_id"] = index
      one_annotations["bbox"] = [float(pp) for pp in p]
      one_annotations["category_id"] = 0

      all_annotations_truth.append(one_annotations)
      count_truth += 1

      truths_boxes.append([x1,y1,x2,y2])

  if problem:
    continue

  if problem:
    print("wtf")

  one_image_truth["id"] = index
  one_image_truth["file_name"] = only_name
  one_image_truth["width"] = image_dict["width"]
  one_image_truth["height"] = image_dict["height"]

  all_image_truth.append(one_image_truth)
  

  # ADD PREDS
  im = cv2.imread(image_name)
  outputs = predictor(im)
  predictions_detectron = outputs["instances"].pred_boxes.tensor.cpu().numpy()
  scores_detectron = outputs["instances"].scores.cpu().numpy()

  for j, p in enumerate(predictions_detectron):
    #if scores_detectron[j]>threshold:
    x1 = p[0]
    y1 = p[1]
    x2 = p[2]
    y2 = p[3]
    w = x2-x1
    h = y2-y1
    #x2 = p[0] + w
    #y2 = p[1] + h
    
    one_annotations = {}
    one_annotations["id"] = count_pred
    one_annotations["segmentation"] = []
    one_annotations["area"] = float(w * h)
    one_annotations["iscrowd"] = 0
    one_annotations["ignore"] = 0
    one_annotations["image_id"] = index
    one_annotations["bbox"] = [float(pp) for pp in [x1,y1,w,h]]
    one_annotations["category_id"] = 0
    one_annotations["score"] = float(scores_detectron[j])

    all_annotations_preds.append(one_annotations)
    count_pred += 1

    preds_boxes.append([x1,y1,x2,y2])

  one_image_pred["id"] = index
  one_image_pred["file_name"] = only_name
  one_image_pred["width"] = image_dict["width"]
  one_image_pred["height"] = image_dict["height"]

  all_image_preds.append(one_image_pred)

  result = metric_table_bank_union(np.array(truths_boxes), np.array(preds_boxes))
  numerator, denominator_precision, denominator_recall, old_way_area_union = result

  sum_numerator += numerator
  sum_numerator_old += old_way_area_union
  sum_denominator_precision += denominator_precision
  sum_denominator_recall += denominator_recall


# ADD TRUTHS
test_latex_truth["images"] = all_image_truth
test_latex_truth["annotations"] = all_annotations_truth
test_latex_truth["categories"] = [{"id": 0, "name": "table", "supercategory": "none"}]

test_type = "word-latex-publaynet"
train_type = "latex"

path_truth = "prediction_and_truth_models/train_"+train_type+"/annotations_test_"+test_type+"/truth"
if not path.exists(path_truth):
  os.makedirs(path_truth)
out_file = open(path_truth + "/test_"+test_type+"_truth.json", "w")
json.dump(test_latex_truth, out_file, indent = 6)
out_file.close()
  
# ADD PREDS
test_latex_predictions["images"] = all_image_preds
test_latex_predictions["annotations"] = all_annotations_preds
test_latex_predictions["categories"] = [{"id": 0, "name": "table", "supercategory": "none"}]

path_pred = "prediction_and_truth_models/train_"+train_type+"/annotations_test_"+test_type+"/preds"
if not path.exists(path_pred):
  os.makedirs(path_pred)
out_file = open(path_pred + "/test_"+test_type+"_preds.json", "w")
json.dump(test_latex_predictions, out_file, indent = 6)
out_file.close()

precision = sum_numerator/sum_denominator_precision
recall = sum_numerator/sum_denominator_recall
f1 = (2 * precision * recall) / (precision + recall)

print("First")
print("Precision ", precision)
print("Recall ", recall)
print("F1 ", f1)

# Make Precision-Recall curve

In [ ]:
test_data = get_test_data_both()
#threshold_pred = [0.0, 0.6, 0.8, 0.95, 0.97, 0.99]

threshold_pred = [0.975, 0.98, 0.985]

recall_with_threshold = []
precision_with_threshold = []

for threshold_ind, threshold in enumerate(threshold_pred):
  print(threshold)

  precision_tab = []
  recall_tab = []

  sum_numerator = 0
  sum_numerator_old = 0
  sum_denominator_precision = 0
  sum_denominator_recall = 0

  test_count = 0

  for i in range(len(test_data)):
    if i%4000 == 0:
      print(i)
    image_dict = test_data[i]
    image_name = "/data/rali5/Tmp/yockelle/TableBank/" + image_dict["file_name"]
    im = cv2.imread(image_name)
    outputs = predictor(im)

    predictions_detectron = outputs["instances"].pred_boxes.tensor.cpu().numpy()
    scores_detectron = outputs["instances"].scores.cpu().numpy()
    predictions_with_threshold = []
    for j, p in enumerate(predictions_detectron):
      if scores_detectron[j]>threshold:
        predictions_with_threshold.append(p)
    predictions_with_threshold = np.array(predictions_with_threshold)

    # Get the ground truth
    bbox_ground_truth = []
    truth = image_dict["annotations"]
    for t in truth:
        x_min = t["bbox"][0]
        y_min = t["bbox"][1]
        w = t["bbox"][2]
        h = t["bbox"][3]
        x_max = x_min + w
        y_max = y_min + h
        bbox_ground_truth.append([x_min, y_min, x_max, y_max])

    result = metric_table_bank_union(bbox_ground_truth, predictions_with_threshold)

    if result != None:
      test_count += 1
      numerator, denominator_precision, denominator_recall, old_way_area_union = result

      #precision_tab.append(precision)
      #recall_tab.append(recall)

      sum_numerator += numerator
      sum_numerator_old += old_way_area_union
      sum_denominator_precision += denominator_precision
      sum_denominator_recall += denominator_recall

  precision = sum_numerator/sum_denominator_precision
  recall = sum_numerator/sum_denominator_recall
  f1 = (2 * precision * recall) / (precision + recall)

  precision_with_threshold.append(precision)
  recall_with_threshold.append(recall)

  #precision_old = sum_numerator_old/sum_denominator_precision
  #recall_old = sum_numerator_old/sum_denominator_recall
  #f1_old = (2 * precision_old * recall_old) / (precision_old + recall_old)

  print("precision ", precision)
  print("recall ", recall)
  print("f1 ", f1)

with open('X101model_recall_with_threshold_latex-word_0.975_0.98_0.985.pkl', 'wb') as f:
  pickle.dump(recall_with_threshold, f)

with open('X101model_precision_with_threshold_latex-word_0.975_0.98_0.985.pkl', 'wb') as f:
    pickle.dump(precision_with_threshold, f)


In [ ]:
with open('image_curve/v2_recall_with_threshold_latex-word.pkl', 'rb') as f:
    recall_with_threshold = pickle.load(f)

with open('image_curve/v2_precision_with_threshold_latex-word.pkl', 'rb') as f:
    precision_with_threshold = pickle.load(f)

In [ ]:
recall = recall_with_threshold[6]
precision = precision_with_threshold[6]

print((2 * precision * recall) / (precision + recall))

In [ ]:
with open('image_curve/X101model_recall_with_threshold_latex-word.pkl', 'rb') as f:
    recall_with_threshold = pickle.load(f)

with open('image_curve/X101model_precision_with_threshold_latex-word.pkl', 'rb') as f:
    precision_with_threshold = pickle.load(f)

threshold_pred = [0.0, 0.6, 0.8, 0.95, 0.97, 0.975, 0.98, 0.985, 0.99]

plt.figure(figsize=(13, 9))

plt.rcParams['text.usetex']=True
plt.rc({
    "font.family": "serif",
    "font.serif": ["Palatino"]})

colors_choices = ["#f81aa1", "#3cc84b", "#6a2020", "#ff8d00", "#ff0000", "#bc21e9", "#1798e7", "#09701d", "#2af0f0"]

plt.plot(recall_with_threshold, precision_with_threshold, '--', color="k")

for thresh_ind, threshold in enumerate(threshold_pred):
  plt.plot(recall_with_threshold[thresh_ind], precision_with_threshold[thresh_ind], 'o', markersize=10, color=colors_choices[thresh_ind], label=threshold)

plt.plot(0.8817, 0.9670, 'o', markersize=10, color="k", label="paper")

plt.grid()
plt.xlabel('Recall', fontsize=15)
plt.ylabel('Precision', fontsize=15)
plt.xticks(fontsize = 17)
plt.yticks(fontsize = 17)
#plt.xlim(min(recall_with_threshold)-0.009, 1.0)
#plt.ylim(min(precision_with_threshold)-0.02, 1.0)
plt.legend(prop={'size': 14}, title="Prediction threshold", title_fontsize=14)
plt.savefig("X101model-precision_latex-word.png")
plt.show()


In [ ]:
test_data = get_test_data_both()

precision_tab = []
recall_tab = []

sum_numerator = 0
sum_denominator_precision = 0
sum_denominator_recall = 0

test_count = 0

for i in  range(len(test_data)):
  if not i%100:
    print(i)

  image_dict = test_data[i]
  image_name = "/data/rali5/Tmp/yockelle/TableBank/" + image_dict["file_name"]
  im = cv2.imread(image_name)
  outputs = predictor(im)

  predictions_detectron = outputs["instances"].pred_boxes.tensor.cpu().numpy()

  # Get the ground truth
  bbox_ground_truth = []
  truth = image_dict["annotations"]
  for t in truth:
      x_min = t["bbox"][0]
      y_min = t["bbox"][1]
      w = t["bbox"][2]
      h = t["bbox"][3]
      x_max = x_min + w
      y_max = y_min + h
      bbox_ground_truth.append([x_min, y_min, x_max, y_max])

  result = metric_table_bank_union(bbox_ground_truth, predictions_detectron)

  if result != None:
    test_count += 1
    precision, recall, numerator, denominator_precision, denominator_recall = result

    #precision_tab.append(precision)
    #recall_tab.append(recall)

    sum_numerator += numerator
    sum_denominator_precision += denominator_precision
    sum_denominator_recall += denominator_recall

precision = sum_numerator/sum_denominator_precision
recall = sum_numerator/sum_denominator_recall
f1 = (2 * precision * recall) / (precision + recall)
print(sum_numerator)
print(sum_denominator_precision)
print(sum_denominator_recall)